In [1]:
# Import the model code and all the libraries bcs they are so annoying otherwise
from LSTM import LyricsTokenizer, LyricsGenreDataset, LSTMLyrics_by_Genre, collate_fn, generate_lyrics, train_model
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import trange
import string

In [2]:
import re
import os

id_pattern = re.compile(r'^[a-zA-Z0-9]{22},')

current_song_lyrics = []
first_id = False
first_line = True

all_songs = []
all_genres = []

folder_path = 'C:/Users/Chloe/Documents/Spring 2025/CSCI3832_FinalProject-main/LSTM_data'

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    genre = re.sub(r'\d+', '', os.path.splitext(filename)[0])
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if first_line == True:
                first_line = False
                continue
            else:
                if id_pattern.match(line):
                    if first_id == False:
                        first_id = True
                    else:
                        lyrics_in_song = ''.join(current_song_lyrics).strip()
                        if len(lyrics_in_song) >= 10:
                            all_songs.append(lyrics_in_song)
                            all_genres.append(genre) 
                        current_song_lyrics = []
                else:
                    current_song_lyrics.append(line)  

        if current_song_lyrics:
            lyrics_in_song = ''.join(current_song_lyrics).strip()
            if len(lyrics_in_song) >= 10:
                all_songs.append(lyrics_in_song)   
                all_genres.append(genre)  
#remove punctuation
all_songs = [song.translate(str.maketrans('', '', string.punctuation)) for song in all_songs]
# #regular expression from geeks for geeks for removing non-eng char: https://www.geeksforgeeks.org/python-remove-non-english-characters-strings-from-list/
# all_songs = list(filter(lambda ele: re.search("[a-zA-Z]+", ele) is not None, all_songs))
# Build tokenizer
tokenizer = LyricsTokenizer(min_freq=1, max_length=100)#max length is 100 tokens for computing efficiency
tokenizer.build_vocab(all_songs)
tokenized_lyrics = tokenizer.tokenize(all_songs)

# Genre -> index mapping
genre2idx = {genre: idx for idx, genre in enumerate(sorted(set(all_genres)))}

# Dataset + Dataloader
dataset = LyricsGenreDataset(tokenized_lyrics, all_genres, genre2idx)

Missed 6119 out of 985307 words -- 0.62%


In [50]:
# Re-create the model
vocab_size = tokenizer.vocab_size()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_genres = len(genre2idx)


model = LSTMLyrics_by_Genre(
    vocab_size=vocab_size,
    embed_dim=256,
    hidden_size=256,
    genre_embed_size=32,
    num_layers=1,
    bidirectional=False,
    teacher_forcing_ratio=0.5,
    num_genres=num_genres
).to(device)

# Load saved weights
model.load_state_dict(torch.load('C:/Users/Chloe/Documents/Spring 2025/CSCI3832_FinalProject-main/src/model/LSTM_genre_no_punct.pth',weights_only=True))
model.eval()  # Put it in eval mode if you're generating or evaluating

lyrics = generate_lyrics(model, tokenizer, "rock", genre2idx=genre2idx, max_len=150, device=device)
print(lyrics)

fighter reasons all right thing no more but you turned on the same but i went to but i is you make you smile i you had the cage and i you didnt know what i believe in too much id even remember if you know you til that way of way that i and i go by the youre when i got down ive got tell me til that shrink but i i wasnt i on what you you stay i everything wont see you i you song on that we got ya has always speak for running feelin wait a each nights on home tonight through me maybe i i all so when i would you i by you own i you you can see you without the face i love myself my love i way when the disappear under it cant stop where it work ill be
